# Separation
In this notebook we look at how gravity separates a mixture of two kinds of discs of differening masses. The discs are initially mixed and start with the same speed (but travelling in differing directions).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
from matplotlib.patches import Circle
import billiards as bl

In [ ]:
# Setup the simulation
L = 300.0  # Simulation width
bottom_left = np.array([-L/2, -L/2])
top_right = np.array([L/2, L/2])

sim = bl.PySim(bottom_left, top_right, 149, 149)

# Set gravity
sim.g = np.array([0.0, -0.1])

# Number, speed, masses, radii of discs
N_discs = 10_000
v = 1.0
R = 1.0

# Generate the masses of the discs
N1 = 5_000
N2 = N_discs - N1
m1, m2 = 1.0, 10.0
m = np.concatenate((np.full(N1, m1), np.full(N2, m2)))

rng = np.random.default_rng(10)
rng.shuffle(m)

sim.add_random_discs(bottom_left, top_right, N_discs, m, R, v=v, pos_allocation='grid')

Create a plot showing initial locations of all discs

In [ ]:
def create_state_plot(cur_pos):
    fig = plt.figure(figsize=(8, 8))
    ax = plt.gca()
    
    plt.xlim(bottom_left[0], top_right[0])
    plt.ylim(bottom_left[1], top_right[1])

    colors = ['tab:blue', 'tab:orange']
    handles = []
    alpha = 0.5    
    
    for mass, color in zip([m1, m2], colors):
        ellipse_col = EllipseCollection(widths=2*R, 
                                    heights=2*R, 
                                    angles=0.0, 
                                    units='x',
                                    offsets=cur_pos[m==mass],
                                    offset_transform=ax.transData, 
                                    alpha=alpha,
                                    facecolor=color)

        handles.append(Circle((0, 0), color=color, alpha=alpha, label=f"m={mass}"))

        ax.add_collection(ellipse_col)
    
    plt.legend(handles=handles, loc='upper right')

    return ax

In [ ]:
initial_pos = sim.initial_state['r']
_ = create_state_plot(initial_pos)

Now look at the state after some time has passed. Note this may take 10-15 minutes to compute.

In [ ]:
# Run the simulation for a bit, only interested in the final state, so no need to record events
sim.advance(75_000_000, 10000.0, False)
print(f"Simulation ended at {sim.current_time}")

In [ ]:
cur_state = sim.current_state
_ = create_state_plot(cur_state['r_cor'])

We can now look how the number of each species varies with height.

In [ ]:
bins = np.linspace(-L/2, L/2, 11)
disc_pos = cur_state['r_cor']

fig, axs = plt.subplots(2, 1, sharex=True)
fig.subplots_adjust(hspace=0)


N_disc_y = []

for m_ind, mass in enumerate([m1, m2]):
    n, _, _ = axs[m_ind].hist(disc_pos[m==mass, 1], bins)
    N_disc_y.append(n)

plt.xlabel("$y$")
axs[0].set(ylabel="$n_1$")
axs[1].set(ylabel="$n_2$")


plt.show()

In [ ]:
total = N_disc_y[0] + N_disc_y[1]
above = N_disc_y[0] / total
below = N_disc_y[1] / total

pos_dict = {
    m1: above,
    m2: below,
}

In [ ]:
fig, ax = plt.subplots()
bottom = np.zeros(bins.shape[0]-1)

for mass, weights in pos_dict.items():
    ax.bar(bins[:-1]+20.0, weights, width=40.0, label=mass, bottom=bottom)
    bottom += weights

plt.xlabel("y")
plt.ylabel("Fraction of each species")
plt.legend(loc='lower right')
plt.show()